In [71]:
from os import listdir
from collections import Counter, defaultdict
import plotly

from bs4 import BeautifulSoup
import requests
import pandas as pd


def _read_file():
    with open("followers.html", "r", encoding="utf-8") as f:
        html = f.read()
    soup = BeautifulSoup(html, features="html.parser")
    return soup


In [72]:
def _extract_follower_images():
    soup = _read_file()

    all_a_elements = soup.find_all("a", href=True)
    all_links = [a["href"] for a in all_a_elements]
    all_links_unique = list(Counter(all_links).items())
    all_followers = [url[1:-1] for url, _ in all_links_unique]

    all_img_elements = soup.find_all("img", src=True)
    all_imgs = [a["src"] for a in all_img_elements]

    follower_images = dict(zip(all_followers, all_imgs))

    return follower_images


follower_images = _extract_follower_images()

In [73]:
if len(listdir('images')) != len(follower_images):
    for follower, url in follower_images.items():
        response = requests.get(url)
        open(f"./images/{follower}.jpg", "wb").write(response.content)

In [74]:
def _extract_total_engagement():
    data_files = [file for file in listdir('posts') if not file.startswith('_')]
    posts = []
    for file in data_files:
        filename = file.split('.')[0]
        exec(f'from posts.{filename} import p; posts.append(p)')

    total_engagement = {follower: sum([post[follower] for post in posts]) for follower in follower_images}
    return total_engagement


follower_engagement = _extract_total_engagement()

In [75]:
follower_engagement_reversed = defaultdict(list)
for follower, engagement_score in follower_engagement.items():
    follower_engagement_reversed[engagement_score].append(follower)

follower_engagement_reversed = {k: follower_engagement_reversed[k] for k in sorted(follower_engagement_reversed, reverse=True)}
follower_engagement_reversed

{5: ['unbothered.moisturized'],
 4: ['jd_is_frostyy',
  'eli_salted',
  'keenart_',
  'priyashnil',
  'marvyp97',
  'salamitsesther'],
 3: ['kayleejune_',
  'dheie8a',
  'harsimran__panda',
  'willharinga',
  'korinerosario',
  'asinveryyoung',
  'hashtags_g',
  'juliaalee'],
 2: ['vagittarius',
  'sukh.maanix',
  'iriskahh',
  'e__zhang',
  'aryanlobie',
  'disposab.le',
  'puneeth_david',
  'patwal69',
  'firstling_',
  'wes.tavares',
  'jayfujiirajani'],
 1: ['dean_phommahaxay',
  '2hard2hold',
  'jashanz',
  'megan_liew',
  'navjotk___',
  'findingmemyselfand_',
  'arshveenhora',
  'ab00si',
  'norman__tan',
  'penguye',
  'hubbard.rosie',
  'byunie96',
  'camlindsay98',
  'royspires',
  'emsuns',
  'alex___kimmy',
  'owen_schonewille'],
 0: ['rikerus',
  'adventuresofbooplesnoot',
  'amy_ff_h',
  'odd_az',
  'maikamontgomery',
  'stolenlighter',
  'claudiamckechnie',
  'sophiologin',
  'ebbastenhaug',
  'troy.matich',
  'jasmeersingh',
  'gao.runzhe',
  'akbloomfield',
  'shafalov

In [98]:
import plotly.graph_objects as go


colours = ['#FF0E00', '#FF7B1D', '#FFD839', '#EDFF56', '#B7FF72', '#96FF8F']
colours.reverse()

data = {
    'engagement': ['80-100%', '60-80%', '40-60%', '20-40%', '1-20%', '0%'],
    '': ['', '', '', '', '', '<br></br> <br></br> <br></br> <br></br> <br></br>'],
    'colour': colours
}
df = pd.DataFrame(data)


fig = go.Figure(data=[go.Table(
  columnorder = [1,2],
  columnwidth = [14,100],
  header = dict(
    values = [['Engagement'], ['']],
    line_color='darkslategray',
    fill_color=['lightgray'],
    align=['center','center'],
    font=dict(color='black', size=18),
    height=40
  ),
  cells=dict(
    values=[df['engagement'], df['']],
    line_color='darkslategray',
    fill_color=[df.colour, 'white'],
    align=['center', 'center'],
    font_size=18,
    height=80)
    )
])

y = 0.9
y_dec = 0.095
x_init = 0.15
x_inc = 0.05
for score, follower_list in follower_engagement_reversed.items():
    x = x_init    
    for i, follower in enumerate(follower_list):
        src = f'./images/{follower}.jpg'
        fig.add_layout_image(
            dict(
                source=src,
                x=x,
                y=y,
            ))
        x += x_inc
        if x > 1 and (i+1) != len(follower_list):
            x = x_init
            y -= y_dec
    y -= y_dec
    
        

fig.update_layout_images(dict(
        xref="paper",
        yref="paper",
        sizex=0.05,
        sizey=0.05,
        xanchor="center",
        yanchor="middle"
))

fig.update_layout(
    title_text="@joshwascool follower engagement tier list",
    height=1000,
    width=1200,
    template="plotly_white",
    font_size=20
)
fig.show()